Will follow [Huggingface's translation tutorial](https://huggingface.co/docs/transformers/tasks/translation) more or less.

In [ ]:
# Stuff for running the same notebook locally and on Google Colab for training
import sys
IN_COLAB = 'google.colab' in sys.modules
fp = "/content/drive/MyDrive/Colab Notebooks/ersetzBERT/training_data_gender.jsonl" if IN_COLAB else "../data/training_data_gender.jsonl"
if IN_COLAB:
    from google.colab import drive # type: ignore
    drive.mount('/content/drive')
    %pip install transformers datasets sacrebleu sentencepiece

In [ ]:
from datasets import load_dataset

data = load_dataset("json", data_files=fp)
data = data["train"].train_test_split(test_size=0.2, shuffle=False)
data["train"][:3]

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [ ]:
def preprocess_function(data):
    inputs = [f"""Ersetze "{a}" durch "{b}": {x}""" for a, b, x in zip(data["a"], data["b"], data["x"])]
    print(inputs)
    targets = data["y"]
    model_inputs = tokenizer(inputs)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

In [ ]:
from transformers import TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    log_folder = "logs"
    %tensorboard --logdir={log_folder}

In [ ]:
if IN_COLAB:
    from tensorflow.keras.callbacks import TensorBoard

    history = model.fit(
        x=tf_train_set, validation_data=tf_test_set, epochs=20, callbacks=[TensorBoard(log_dir=log_folder)]
    )


In [ ]:
if IN_COLAB:
    model.save("/content/drive/MyDrive/Colab Notebooks/ersetzBERT/model")

In [ ]:
from transformers import pipeline

generator = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

In [ ]:
entries = list(zip(data["test"]["x"], data["test"]["a"], data["test"]["b"], data["test"]["y"]))

In [ ]:
for x, a, b, y in entries[:5]:
    prompt = f"""Ersetze "{a}" durch "{b}": {x}"""
    print(prompt)
    prediction = generator(prompt)[0]["generated_text"]
    print(prediction)
    print()

In [ ]:
examples = """
Ersetze "Autofahrer" durch "Mensch, der Auto fährt": Heute morgen im Stau haben mich die Autofahrer wieder sehr aufgeregt.
Ersetze "Behinderter" durch  "Mensch mit Behinderungen": Behinderte mit entsprechenden Ausweisen bekommen ermäßigten Eintritt.
Ersetze "Student" durch "studierende Person" bzw. "Studierende": Viele faule Studenten studieren gar nicht wirklich.
Ersetze "Student" durch "studierende Person" bzw. "Studierende": Maria ist kein Student.
Ersetze "Lehrer" durch "Lehrerin oder Lehrer" bzw. "Kollegium": Die Lehrer machen morgen einen Ausflug.
Ersetze "Lehrer" durch "Lehrerin oder Lehrer" bzw. "Kollegium": Ein promovierter Mathelehrer ist noch nie im Unterricht eingeschlafen.
Ersetze "Polizist" durch "Polizistin oder Polizist": Die Polizisten machen oft Überstunden.
Ersetze "Gaul" durch "Stute oder Gaul": Einem geschenkten Gaul schaut man nicht ins Maul.
""".strip().split("/n")
for prompt in examples:
    print(prompt)
    prediction = generator(prompt)[0]["generated_text"]
    print(prediction)
    print()